In [1]:
import numpy as np 
import pandas as pd 
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
import os
from sklearn.metrics.pairwise import cosine_similarity
from torchvision.models.resnet import ResNet18_Weights
from torchvision import transforms
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA

In [2]:
def extract_feature_vector(img_tensor):
    # Assume img_tensor is already a preprocessed tensor
    t_img = Variable(img_tensor)

    #Create a zero vector that will hold the feature vector
    # The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(1, 512, 1, 1)

    #Define a function to copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)

    # Attach the function to our selected layer
    h = layer.register_forward_hook(copy_data)

    # Run the model on our transformed image
    model(t_img)

    # Detach our copy function from the layer
    h.remove()

    # Return the feature vector
    return my_embedding.squeeze().numpy()

In [3]:
def process_and_save_features(input_base_folder, output_base_folder):
    folder_counter = 1

    for subdir, dirs, files in os.walk(input_base_folder):
        if not files:  # Skip directories without files
            continue

        all_feature_vectors = []  # Collect feature vectors for this folder

        for file in files:
            if file.lower().endswith('.jpeg'):
                img_path = os.path.join(subdir, file)
                img = Image.open(img_path)
                img = convert_to_3_channels(img)  # Convert to 3 channels if needed
                img_tensor = preprocess(img)  # Apply preprocessing
                img_tensor = img_tensor.unsqueeze(0)  # Add an extra batch dimension

                feature_vector = extract_feature_vector(img_tensor)  # Extract feature vector
                all_feature_vectors.append(feature_vector)

        # 将特征向量转换为 NumPy 数组并使用 PCA 降维
        all_feature_vectors_np = np.array(all_feature_vectors)
        pca = PCA(n_components=30)
        reduced_vectors = pca.fit_transform(all_feature_vectors_np)

        # 为当前文件夹创建输出目录
        current_output_folder = os.path.join(output_base_folder, f"train_feature{folder_counter}")
        if not os.path.exists(current_output_folder):
            os.makedirs(current_output_folder)

        # 保存降维后的特征向量，文件名从1开始计数
        for i, reduced_vector in enumerate(reduced_vectors, start=1):
            output_path = os.path.join(current_output_folder, f"{i}.npy")
            np.save(output_path, reduced_vector)

        folder_counter += 1

In [4]:
# def process_and_save_features(input_base_folder, output_base_folder):
#     folder_counter = 1

#     for subdir, dirs, files in os.walk(input_base_folder):
#         if not files:  # 跳过没有文件的目录
#             continue

#         output_folder = os.path.join(output_base_folder, f"train_feature{folder_counter}")
#         if not os.path.exists(output_folder):
#             os.makedirs(output_folder)

#         counter = 1
#         for file in files:
#             if file.lower().endswith('.jpeg'):
#                 img_path = os.path.join(subdir, file)
#                 img = Image.open(img_path)  # PIL image
#                 img_tensor = preprocess(img)  # Apply preprocessing
#                 img_tensor = img_tensor.unsqueeze(0)  # Add an extra batch dimension

#                 feature_vector = extract_feature_vector(img_tensor)  # Directly use the tensor
#                 feature_vector_np = feature_vector.squeeze()

#                 # Save the feature vector to a file
#                 output_path = os.path.join(output_folder, str(counter) + '.npy')
#                 np.save(output_path, feature_vector_np)

#                 counter += 1

#         folder_counter += 1

In [5]:
def convert_to_3_channels(img):
    """Convert an image to 3 channels if it's not already."""
    if img.mode != 'RGB':
        img = img.convert('RGB')
    return img

In [6]:
# Load the pretrained ResNet18 model using the new weights parameter
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.eval()  # Set to evaluation mode

# Select the layer from which to extract features
layer = model.avgpool
# Normalize transform
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# Define the image preprocessing function
preprocess = transforms.Compose([
    transforms.Lambda(convert_to_3_channels), 
    transforms.Resize(256, antialias=True),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

In [7]:
# 调用函数
input_base_folder = './train'
output_base_folder = './train_feature'
process_and_save_features(input_base_folder, output_base_folder)